# Dive to Normalizing Flows
---

Цели:
- Разобраться с формальным выводом NF
- Подробнее рассмотреть потоки и их свойства
- Деквантизация

Содержание:
- Вывод нормализующих потоков
- Planar Flow
- Affine Coupling Layer
- Другие потоки и их свойства

Ссылки:
- [Variational Inference with Normalizing Flows](https://arxiv.org/pdf/1505.05770v6.pdf)
- [Learning Discrete Distributions by Dequantization](https://arxiv.org/pdf/2001.11235.pdf)
- [DENSITY ESTIMATION USING REAL NVP](https://arxiv.org/pdf/1605.08803v3.pdf)
- [Glow: Generative Flow with Invertible 1x1 Convolutions](https://arxiv.org/pdf/1807.03039v2.pdf)
- [Flow++: Improving Flow-Based Generative Models with Variational Dequantization and Architecture Design](https://arxiv.org/pdf/1902.00275.pdf)

## Вывод нормализующих потоков

Давайте посмотрим еще раз на этот рисунок и попробуем расписать итерационный процесс

<figure>
<img src="https://lilianweng.github.io/posts/2018-10-13-flow-models/normalizing-flow.png" alt="Примеры схем генеративных моделй" style="width:100%">
<figcaption align = "center">Иллюстрация преобразования нормально распределенного z0 в zK из реального  распределения.</figcaption>
</figure>

Положим, существуют
$$
z_{i-1}\sim p_{i-1}(z_{i-1}), ~ z_i = f_i(z_{i-1})\Leftrightarrow z_{i-1} = f^{-1}_i(z_i)
$$

где $z_{i-1}$ — вектор, распределенный как $p_{i-1}$, а $f_i$ — функция, отображающая $z_{i-1}$ в $z_i$. Также существует обратная к $f$ функция $f^{-1}$, которая отображает в обратную сторону.

Переход по графу на рисунке можно расписать как

$$
\begin{split}
p_i(z_i) & \stackrel{(1)}{=} p_{i-1}(f_i^{-1}(z_i))\left|\det\left(\dfrac{df^{-1}_i(z_i)}{dz_i}\right)\right| \\
& \stackrel{(2)}{=} p_{i-1}(z_{i-1})\left|\det\left(\dfrac{df_i(z_{i-1})}{dz_{i-1}}\right)^{-1}\right| \\
& \stackrel{(3)}{=} p_{i-1}(z_{i-1})\left|\det\left(\dfrac{df_i(z_{i-1})}{dz_{i-1}}\right)\right|^{-1} \\
\end{split}
$$

Разберем подробнее каждый переход

1. Получено из теоремы о замене переменной для многомерных случайных величин.
2. Следует из теоремы об обратимых функциях.
3. Для обратимых матриц существует свойство определителя $\det(A^{-1}) = \det(A)^{-1}$.

### Теорема о замене переменной

Положим существует переменная $z$ распределенная как $z\sim\pi(z)$, при помощи которой мы хотим получить новую независимую переменную, используя преобразование $x=f(z)$. Функцию $f$ выберем так, что $\exists f^{-1}: z=f^{-1}(x)$. Вопрос, каким будет распределение $x\sim p(x)$?

Заметим, что если $f(z)$ — дифференцируема на некоторой области и кусочно-монотонна, тогда по свойству преобразования случайной величины
$$
p(x) = \pi(z)\left|\dfrac{dz}{dx}\right|
$$
отсюда получаем, что
$$
\pi(z)\left|\dfrac{dz}{dx}\right| = \pi(f^{-1}(x))\left|\dfrac{df^{-1}}{dx}\right| = \pi(f^{-1}(x))|(f^{-1})^{'}(x)|
$$

В случае многомерной случайной величины выражение схоже

$$
\vec{z}\sim\pi(\vec{z}), ~\vec{x}=f(\vec{z}),~\vec{z}=f^{-1}(\vec{x})
$$

$$
p(\vec{x}) = \pi(\vec{z})\left|det\dfrac{d\vec{z}}{d\vec{x}}\right| = \pi(f^{-1}(\vec{x}))\left|det\dfrac{df^{-1}(\vec{x})}{d\vec{x}}\right|
$$

где $det\frac{df^{-1}(\vec{x})}{d\vec{x}}$ — Якобиан (определитель матрицы Якоби) функции $f^{-1}$.

### Теорема об обратных функциях
Теорема об обратных функциях говорит о том, что если $y = f(x),~x=f^{-1}(y)$, то
$$
\dfrac{df^{-1}(y)}{dy} = \dfrac{dx}{dy}=\left(\dfrac{dy}{dx}\right)^{-1} = \left(\dfrac{df(x)}{dx}\right)^{-1}
$$

### Свойство определителя обратной матрицы
$$
A\cdot A^{-1} = I \Rightarrow \det(A\cdot A^{-1}) = \det(I)  \Rightarrow \det(A\cdot A^{-1}) = 1 \\
\det(A)\cdot \det(A^{-1}) = 1 \Rightarrow \det(A^{-1}) = \dfrac{1}{\det(A)} = \det(A)^{-1}
$$

### Переход к суперпозиции

Таким образом получить из нормально распределенных $z_0$ объекты $x=z_K$ можно через суперпозицию всех функций $f_i$ как
$$
x=z_K=f_K\circ f_{K-1}\circ\cdots\circ f_1(z_0)
$$

Тогда трансформация логарифма плотности распределения
$$
\begin{split}
\log p(x) = \log\pi_K(z_K) & = \log\pi_{K-1}(z_{K-1})-\log\left|\det\dfrac{df_K}{dz_{K-1}}\right| \\
& = \log\pi_{K-2}(z_{K-2}) -\log\left|\det\dfrac{df_{K-1}}{dz_{K-2}}\right| -\log\left|\det\dfrac{df_K}{dz_{K-1}}\right| \\
& = \cdots \\
& = \log\pi_0(z_0) - \sum\limits_{i=1}^K\log\left|\det\dfrac{df_i}{dz_{i-1}}\right|
\end{split}
$$

Эта функция является логарифмом правдоподобия и может быть использована в качестве обучения. Однако, так как оптимизационная задача с функцией правдоподобия — задача максимизации, возьмем отрицательную функцию логаифма правдоподобия (negative log likelyhood loss), которую и будем минимизировать.
$$
NLLLoss = - \log\pi_0(z_0) + \sum\limits_{i=1}^K\log\left|\det\dfrac{df_i}{dz_{i-1}}\right|
$$

Здесь преобразование $z_i=f_i(z_{i-1})$ называется потоком (flow), а полная последовательность из преобразований называется нормализующим потоком (normalizing flow).

Стоит уточнить, что на функции $f_i$ накладывается ряд требований:

1. $f_i$ должна быть легко обратимой функцией
2. Якобиан $\frac{df_i}{dz_{i-1}}$ должен быть легко вычислимым

## Planar Flow

Рассмотрим первую трансформацию, использующуюся в качестве потока — планарный поток. Это семейство функций вида
$$
f(x) = uh(w^Tx + b)
$$
где
- $h$ — функция активации
- $⟨⋅, ⋅⟩$ — cкалярное произведение
- $u, w, b$ — обучаемые параметры

### Обратимость

Проблема этого потока заключается в том, что не каждая функция этого семейства может быть обратима. Для обратимости требуется чтобы
1. $h(x) = \tanh(x)$
2. $w^Tu \geq -1$

Второе условие может быть достигнуто при посощи обновления вектора $u$ так, что
$$
\hat{u}(w, u) = u + \left[m(w^Tu) - w^Tu\right] \frac{w}{||w||_2}
$$
где $m(x) = -1+\log{(1+e^x)}$.

### Якобиан
Якобиан можно записать как
$$
\det{J} = \det{\left(I + u\psi(x)^T\right)} = 1 + u^T\psi(x)
$$
где
- $I$ — единичная матрица
- $\psi(x) = h'(w^Tx + b)$
- $h'$ — произвлдная от $h$




## Affine Coupling Layer
$$
\begin{cases}
y_i = x_i,~\forall i=1,2,\dots,d \\
y_i = x_i \odot \exp(s(x_{i-d})) + t(x_{i-d}),~\forall i=d+1,\dots,D
\end{cases}
$$

где $s(\cdot)$ — масштабирование, $t(\cdot)$ — смещение, а $\odot$ — поэлементное умножение.

Теперь убедимся, что это преобразование подходит нам по критериям функции $f$.

### Обратимость
$$
\begin{cases}
y_i = x_i~,\\
y_i = x_i \odot \exp(s(x_{i-d})) + t(x_{i-d})~;
\end{cases} \Leftrightarrow
\begin{cases}
x_i = y_i~,\\
x_i = (y_i - t(x_{i-d})) \odot \exp(-s(x_{i-d}))~.
\end{cases}
$$

Так как преобразование линейно, оно не требует поиска обратных функций к $s(\cdot)$ и $t(\cdot)$, а значит, что вычисляется быстро.

### Якобиан

Легко заметить, что Якобиан функции имеет вид нижней треугольной матрицы

$$
J =
\begin{bmatrix}
\mathbf{I} & \mathbf{0} \\
\frac{\partial y_{d+1:D}}{\partial x^T_{1:d}} & \text{diag}(\exp[s(x_{1:d})])
\end{bmatrix}
$$

где
- $\mathbf{I}_d$ — единичная матрица, полученная из $\frac{\partial y_{1:d}}{\partial x_{1:d}} = \frac{\partial x_{1:d}}{\partial x_{1:d}}$ размерности $d\times d$
- $\mathbf{0}$ — нулевая матрица, полученная из $\frac{\partial y_{1:d}}{\partial x_{d+1:D}} = \frac{\partial x_{1:d}}{\partial x_{d+1:D}}$ размерности $d\times (D-d)$
- $\frac{\partial y_{d+1:D}}{\partial x^T_{1:d}}$ — минор матрицы Якоби размерности $(D-d)\times d$
- $\text{diag}(\exp[s(x_{1:d})])$ — диагональная матрица, где на диагонали значения $\exp[s(x_{1:d})]$, а вне — нули. Размерность это матрицы $(D-d)\times(D-d)$

Можно заметить, что такая матрица может существовать только при $d = \frac{D}{2}$.

Так как матрица имеет вид нижней треугольно, ее определитель считается по диагонали, а значит, что левую нижнюю часть матрицы знать не обязательно. Тогда Якобиан вычисляется как

$$
\det(J) = \prod\limits_{j=1}^{D-d}\exp(s(x_{1:d}))_j = \exp\left(\sum\limits_{j=1}^{D-d}s(x_{1:d})_j\right)
$$

Так как вычисление определителя сводится к вычислению экспоненты суммы выходов $s(\cdot)$, считаться определитель будет быстро.

Так как к функциям $s(\cdot)$ и $t(\cdot)$ не выставлено никаких сложных требований, в их качестве можно использовать функции класса нейронных сетей.

### Использование маски

Разбиаение преобразований на тривиальные и афинные в Affine Coupling Layer можно представить как бинарную маску $b$. Тогда преобразование можно переписать как
$$
y = b ⊙ x + (1-b) ⊙ (x ⊙ \exp(s(b ⊙ x)) + t(b ⊙ x))
$$

Тогда обратное преобразование получается следующим
$$
x = b ⊙ y + (1 - b) ⊙ ((y - t(b ⊙ y)) ⊙ \exp(-s(b ⊙ y)))
$$

Так как маска влияет на порядок строк и столбцов в матрице Якобиана нужно сказать, что перестановка строк или столбцов в определители влияет только на знак определителя матрицы, а так как для вычисления плотности распределения мы используем модуль определителя, перестановка строк и столбцов на вычисление плотности не влияет.

Таким образом мы можем свести вычисление оперделителя при использовании маски к базовому определению афинного потока выше, переставляя строки матрицы.

Сам Якобиан можно переписать как
$$
\det(J) = \prod(1-b) ⊙ \exp(s(b \odot x)) = \exp\left(\sum(1-b) ⊙ s(b \odot x)\right)
$$

## Деквантизация

Нормализующие потоки полагаются на правило изменения переменных, которое естественным образом определено в непрерывном пространстве. Применение потоков непосредственно к дискретным данным приводит к нежелательным моделям плотности.

Так на рисунке ниже иллюстрируется как нормализующий поток пытается преобразовать нормальное распределение в дискретное распределение прдставленное двумя точками — $(0, 1),~(1, 0)$. Видно, что модель в итоге приводит распределение к некоторой "линии", что никак не сходится с требуемым распределением. Есть несколько вариантов решения проблемы:
- Решать задачу для дискретного распределения
- Придумать поток, который бы приводил диксретное распределение к непрерывному и наоборот

<figure>
<img src="https://github.com/sswt/dive2gai/blob/unit3/.github/images/simple_flow.png?raw=true" alt="Пример работы нормализующего потока без квантизации с дискретным распределением" style="width:100%">
<figcaption align = "center">Пример работы нормализующего потока без квантизации с дискретным распределением.</figcaption>
</figure>



### Обычная деквантизация

Положим, что у нас есть дискретная переменная $x$ с плотностью $p(x)$. Скажем, что шум $u\sim U(0, 1)^D$ — аддитивный шум из многомерного равномерного распределения и есть результат добавления шума $y = x + u$. Для новой величины плотность может быть представлена как
$$
p(y) = \int p(x+u)du = \int\dfrac{q(u|x)}{q(u|x)}p(x+u)du=\int\dfrac{p(x+u)}{q(u|x)}q(u|x)du = \mathbb{E}_{u\sim q(u|x)}\left[\dfrac{p(x+u)}{q(u|x)}\right]
$$

где $q(u|x)$ — на самом деле является плотностью $U(0, 1)^D$, так как $u$ не зависит от $x$, то есть $q(u|x) = q(u) = U(0, 1)^D$. Тогда мы можем переписать плотность как
$$
p(y) = \mathbb{E}_{u\sim U(0, 1)^D}\left[p(x+u)\right]
$$

Также, так как дискретное распределение может быть в диапазоне сильно отличном от диапазона нормального распределения принято использовать нормализацию на число уникальных дискретных значений N и обратную к сигмоиде функцию $\sigma^{-1}(y)$ для приведения к виду более похожему на нормальное распределение.
$$
\sigma^{-1}(v) = \log{v} - \log{(1-v)}
$$

<figure>
<img src="https://github.com/sswt/dive2gai/blob/unit3/.github/images/dequant_flow.png?raw=true" alt="Пример работы нормализующего потока с простой квантизацией." style="width:100%">
<figcaption align = "center">Пример работы нормализующего потока с простой квантизацией.</figcaption>
</figure>

### Обратимость 

Для обратного потока необходимо проделать следующий шаги:
1. Обратить функцию $\sigma^{-1}(v)$ через $\sigma(v)$
$$
\sigma(v) = \dfrac{1}{1+e^{-v}}
$$
2. Возвращаем диапазон от $0$ до $N$ умножением на $N$, где $N$ — количество дискретных уникальных значений.
3. Обрезаем значения до дискретных округлением.

### Якобиан

Так как в деквантизации несколько перобразований, нужно рассчитать Якобиан каждого и взять сумму.
1. Добавление аддитивного шума $v=x+u$
$$
\det{J} = \det\left(\dfrac{dv}{dx}\right) = \det\left(\dfrac{d(x+u)}{dx}\right) = \det{I} = 1
$$
2. Нормализация на $N$
$$
\det{J} = \det\left(\dfrac{d(v/N)}{dv}\right) = \frac{D}{N}
$$
3. Обратная сигмоида
$$
\det{J} = \frac{d}{dv}\left[\log{v} - \log{(1-v)}\right] = \dfrac{1}{v(1-v)}
$$
Заметим, что рассчет этого Якобиана не влияет на обучение, так как у этого потока нет обучающихся параметров, но рассчитать его правильно может быть полезно для корректного вычисления функции правдоподобия и построения плотности исхожного распределения.

### Вариационная деквантизация

По предыдущему примеру можно видеть, что деквантизация с использованием нормального распределения несет за собой некоторые издержки. Так, например, довольно сложно потоками получить нормальное многомерное распределение, что видно на примере выше. Также нормальное распределение не учитывает изходную частотность элементов распределения, что тоже делает генерацию некорректной в случаях с несбалансированными распределениями.

Что если вместо заданного распределения шума мы обучим дополнительный поток прогназировать шум исходя из природы исходных данных?

С этой целью возьмем из формулы выше $q(u|x)$ и скажем, что теперь это распределение будет задаваться потоком с параметрами $\theta$. Тогда распишем вероятность
$$
p(y) = \int\dfrac{p(x+u)}{q_\theta(u|x)}q_\theta(u|x)du = \mathbb{E}_{u\sim q_\theta(u|x)}\left[\dfrac{p(x+u)}{q_\theta(u|x)}\right]
$$

<figure>
<img src="https://github.com/sswt/dive2gai/blob/unit3/.github/images/var_deq_flow.png?raw=true" alt="Пример работы нормализующего потока с вариационной деквантизацией" style="width:100%">
<figcaption align = "center">Пример работы нормализующего потока с вариационной деквантизацией.</figcaption>
</figure>

Очевидно, что единственное отличие заключается в том, что теперь Якобиан будет рассчитываться как Якобиан выбранного потока $q_\theta(u|x)$.

## Другие потоки и их свойства

Далее рассмотрим потоки, предложенные в модели Glow. Краткое описание можно посмотреть на каринке ниже
<figure>
<img src="https://lilianweng.github.io/posts/2018-10-13-flow-models/glow-table.png" alt="Краткое описание введенных потоков модели Glow" style="width:100%">
<figcaption align = "center">Пример работы нормализующего потока с вариационной деквантизацией.</figcaption>
</figure>

Все эти потоки использются в общем блоке следующего вида
<figure>
<img src="https://lilianweng.github.io/posts/2018-10-13-flow-models/one-glow-step.png" alt="Поток модели Glow с использованием трех потоков" style="width:50%">
<figcaption align = "center">Пример работы нормализующего потока с вариационной деквантизацией.</figcaption>
</figure>